# Catnip

In [2]:
# Dependencies and configuration
import importlib
from src.config import settings, setup_dirs

izutsumiPaths, notIzutsumiPaths = setup_dirs()

## Pre-processing

### Panel extraction

In [ ]:
from modules.coreMPE.src.adenzu_panel.image_processing import panel

_ = panel.extract_panels_for_images_in_folder_recursive(
    input_dir=str(settings.paths.pages_dir),
    output_dir=str(settings.paths.panels_dir),
    split_joint_panels=False,   # maps to --split-joint-panels
    fallback=True              # maps to --fallback
)

### Head crops

In [ ]:
# Extract head crops (YOLOv8_animeface)
from src.headExtraction import anime_extraction_recursive

valid_exts = {".jpg", ".jpeg", ".png"}
panel_paths = sorted(
    [p for p in settings.paths.panels_dir.iterdir() if p.suffix.lower() in valid_exts]
)
num_crops = anime_extraction_recursive()

print(f"Extracted {num_crops} faces")

## Catnip core

### Model building

In [ ]:
import src.embeddingModel
importlib.reload(src.embeddingModel)

from src.embeddingModel import compute_embeddings, build_model, load_embeddings

#### New model

In [ ]:
# Build fresh model without loading weights (for new embedding computation)
embed_model = build_model(settings.params.img_size, settings.paths.crops_dir, load_weights=False)
compute_embeddings(embed_model, settings.paths.crops_dir, settings.params.img_size)

#### Previous model

In [ ]:
# Build embedding model (loads saved weights if available)
embed_model = build_model(settings.params.img_size, settings.paths.crops_dir, load_weights=True)
embs, crop_paths = load_embeddings(settings.paths.embs_dir, settings.paths.crops_dir)

## Outputting

In [ ]:
import src.query
import src.output
importlib.reload(src.query) 
importlib.reload(src.output)

from src.query import izutsumi_query, izutsuminess_rank
from src.output import save_similar_results, char_nearest_neighbor

import matplotlib.pyplot as plt

In [ ]:
crop, index, score, thre = izutsumi_query(settings.paths.embs_dir,
                                        settings.paths.crops_dir,
                                        settings.params.img_size, 
                                        embed_model, 
                                        izutsumiPaths,
                                        notIzutsumiPaths,
                                        similarity_threshold=-1,
                                        alpha=0.5,
                                        mode='max')

In [ ]:
index = izutsuminess_rank(settings.paths.embs_dir, settings.paths.crops_dir, embed_model, izutsumiPaths, notIzutsumiPaths)

In [ ]:
plt.figure(figsize=(15, 5))
plt.bar(range(len(index)), score)
plt.xlabel("Izutsuminess")
plt.ylabel("Ranked Results")
plt.title("Embed score")
plt.show()

In [ ]:
cutoff = 0

results = char_nearest_neighbor(crop, index[cutoff:], score[cutoff:], thre)

In [ ]:
save_similar_results(crop, index, settings.paths.output_dir, score)